In [13]:
import openbabel
import os
import pandas as pd
from tqdm import tqdm

In [15]:
if 'ygong' in os.getcwd():
    filepath = "../data"
    dir_out = "../output"
else:
    filepath = "/home/gong/Documents/Kaggle_July2019/data"
    dir_out = "/home/gong/Documents/Kaggle_July2019/output"

def load_data(filepath):
    train = pd.read_csv(os.path.join(filepath, 'train.csv'))
    test = pd.read_csv(os.path.join(filepath, 'test.csv'))
    structures = pd.read_csv(os.path.join(filepath, 'structures.csv'))

    print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0], train.shape[1]))
    print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0], test.shape[1]))
    print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0], structures.shape[1]))
    print('\n')

    return train, test, structures

train, test, structures = load_data(filepath)

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Structures dataset shape is  -> rows: 2358657 cols:6




In [8]:
x = structures.groupby('molecule_name').atom_index.max().reset_index(drop=False)
x.columns = ['molecule_name','totalatoms']
x.totalatoms+=1
train = train.merge(x,on='molecule_name')

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,totalatoms_x,totalatoms_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,5,5
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,5,5
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,5,5
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,5,5
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,5,5
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,5,5
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,5,5
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,5,5
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,5,5
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,5,5


In [16]:
obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")
structdir='../data/structures/'
mols=[]
mols_files=os.listdir(structdir)
mols_index=dict(map(reversed,enumerate(mols_files)))
for f in mols_index.keys():
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, structdir+f) 
    mols.append(mol)

In [ ]:
moles.head()

In [17]:
stats = []
for m,groupdf in tqdm(train.groupby('molecule_name')):
    mol=mols[mols_index[m+'.xyz']]
    for i in groupdf.index.values:
        totalatoms = groupdf.loc[i].totalatoms
        firstatomid = int(groupdf.loc[i].atom_index_0)
        secondatomid = int(groupdf.loc[i].atom_index_1)
        entrystats = {}
        entrystats['totalatoms'] = totalatoms
        entrystats['scalar_coupling_constant'] = float(groupdf.loc[i].scalar_coupling_constant)
        entrystats['type'] = groupdf.loc[i]['type']
        a = mol.GetAtomById(firstatomid)
        b = mol.GetAtomById(secondatomid)
        entrystats['molecule_name'] = m
        entrystats['atom_index_0'] = firstatomid
        entrystats['atom_index_1'] = secondatomid
        entrystats['bond_distance'] = a.GetDistance(b)
        entrystats['bond_atom'] = b.GetType()

        #Put the tertiary data in order of distance from first hydrogen
        tertiarystats = {}
        for j,c in enumerate(list(set(range(totalatoms)).difference(set([firstatomid,secondatomid])))):
            tertiaryatom = mol.GetAtomById(c)
            tp = tertiaryatom.GetType()
            dist = a.GetDistance(tertiaryatom)
            ang = a.GetAngle(b,tertiaryatom)*math.pi/180
            while(dist in tertiarystats):
                dist += 1e-15
                # print('Duplicates!',m,j,dist)
            tertiarystats[dist] = [tp,dist,ang]
        
        for k, c in enumerate(sorted(tertiarystats.keys())):
            entrystats['tertiary_atom_'+str(k)] = tertiarystats[c][0]
            entrystats['tertiary_distance_'+str(k)] = tertiarystats[c][1]
            entrystats['tertiary_angle_'+str(k)] = tertiarystats[c][2]
        stats.append(entrystats)

  0%|          | 0/85003 [00:00<?, ?it/s]


AttributeError: 'Series' object has no attribute 'totalatoms'